In [1]:
# %pip install --upgrade langsmith

import os

print("Langchain key to start tracing")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "pr-aching-poisoning-52"
os.environ["LANGCHAIN_API_KEY"] = "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"

Langchain key to start tracing


In [2]:
# %pip install --upgrade langchain_aws

In [1]:
from getpass import getpass
import boto3

AWS_ACCESS_KEY = getpass("AWS Acces key: ")
AWS_SECRET_KEY = getpass("AWS Secret key: ")
AWS_REGION = "ap-south-1"

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
)

In [2]:
from langchain_aws import BedrockLLM

llm = BedrockLLM(client=bedrock_client,model_id="amazon.titan-text-express-v1")

c:\Users\361DT\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_arn" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [37]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
# bs4_strainer = bs4.SoupStrainer(class_=("the-python-tutorial"))
loader = WebBaseLoader(
    web_paths=("https://www.ibm.com/think/topics/data-science",),
    # bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

18037

In [36]:
docs

[Document(metadata={'source': 'https://www.ibm.com/think/topics/data-science', 'title': 'What is Data Science? | IBM', 'description': 'Data science is a multidisciplinary approach to gaining insights from an increasing amount of data. IBM data science products help find the value of your data.', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\nWhat is Data Science? | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                        \n\n\n\n  \n    What is data science?\n\n\n\n\n\n\n    \n\n\n                    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                    Analytics\n                                \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                Link copied\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n        What is data science?\r\n    \n\n\n\nData science combines math an

In [38]:
print(docs[0].page_content[:200])










What is Data Science? | IBM













































































                        



  
    What is data science?






    


                


In [39]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=150, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

55

In [40]:
all_splits

[Document(metadata={'source': 'https://www.ibm.com/think/topics/data-science', 'title': 'What is Data Science? | IBM', 'description': 'Data science is a multidisciplinary approach to gaining insights from an increasing amount of data. IBM data science products help find the value of your data.', 'language': 'en', 'start_index': 9}, page_content='What is Data Science? | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                        \n\n\n\n  \n    What is data science?\n\n\n\n\n\n\n    \n\n\n                    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                    Analytics\n                                \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                Link copied\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n        What is data science?'),
 Document(metadata={'source': 'https://ww

In [41]:
from langchain_chroma import Chroma
from langchain_aws.embeddings.bedrock import BedrockEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=BedrockEmbeddings(client=bedrock_client,model_id="amazon.titan-embed-text-v2:0"))

In [14]:
vectorstore

In [44]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

retrieved_docs = retriever.invoke("difference between data science and cloud computing")

len(retrieved_docs)

2

In [45]:
print(retrieved_docs[0].page_content)
print('---------')
print(retrieved_docs[1].page_content)


Data science and cloud computing
---------
Cloud computing scales data science by providing access to additional processing power, storage, and other tools required for data science projects.
